In [1]:
import pandas as pd
import pathlib

In [2]:
curr_dir = pathlib.Path().resolve()
print(curr_dir)

/home/inductive-anks/yield-prediction/Yield-Prediction/Code Files/Tehsil-Village-Mapping


In [3]:
home_dir = curr_dir.parent.parent

In [4]:
village_list_df = pd.read_csv(home_dir.as_posix() + '/data/Final Data/Final-Merged-Data(2008-2011 to 2017-2020).csv')

### Number of Tehsils greater than one Village

In [5]:
grouped = village_list_df.groupby(['State', 'Year'])['Tehsil Name'].nunique().reset_index()

In [6]:
grouped

,State,Year,Tehsil Name
0,andhra pradesh,2008-2011,43
1,andhra pradesh,2011-2014,60
2,andhra pradesh,2014-2017,60
3,andhra pradesh,2017-2020,59
4,assam,2008-2011,46
...,...,...,...
68,uttarakhand,2017-2020,4
69,west bengal,2008-2011,53
70,west bengal,2011-2014,53
71,west bengal,2014-2017,55


In [7]:
total_unique_tehsils_per_year = grouped.groupby('Year')['Tehsil Name'].sum().reset_index()
total_unique_tehsils_per_year.columns = ['Year', 'Total Number of unique Tehsils']

In [8]:
total_unique_tehsils_per_year

,Year,Total Number of unique Tehsils
0,2008-2011,719
1,2011-2014,683
2,2014-2017,791
3,2017-2020,787


In [9]:
multi_village_tehsils = village_list_df.groupby(['Tehsil Name', 'Year'])['Village Name'].nunique().reset_index()
multi_village_tehsils = multi_village_tehsils[multi_village_tehsils['Village Name'] > 1]
multi_village_tehsils_per_year = multi_village_tehsils.groupby('Year')['Tehsil Name'].nunique().reset_index()
multi_village_tehsils_per_year.columns = ['Year', 'Total Number of unique Tehsils > 1 village']

In [10]:
multi_village_tehsils_per_year

,Year,Total Number of unique Tehsils > 1 village
0,2008-2011,167
1,2011-2014,125
2,2014-2017,174
3,2017-2020,153


In [11]:
final_tehsil_data = pd.merge(total_unique_tehsils_per_year, multi_village_tehsils_per_year, on='Year', how='left').fillna(0)

In [12]:
final_tehsil_data.head()

,Year,Total Number of unique Tehsils,Total Number of unique Tehsils > 1 village
0,2008-2011,719,167
1,2011-2014,683,125
2,2014-2017,791,174
3,2017-2020,787,153


In [13]:
final_tehsil_data_multiple_village = village_list_df[village_list_df['Tehsil Name'].isin(multi_village_tehsils['Tehsil Name'])]

In [14]:
final_tehsil_data_multiple_village.shape

(1964, 7)

In [15]:
final_tehsil_data_multiple_village.head()

,State,Zone,District Name,Tehsil Code,Tehsil Name,Village Name,Year
43,assam,i,dhemaji,1,dhemaji,na-pam,2008-2011
44,assam,i,dhemaji,1,dhemaji,dushuti,2008-2011
45,assam,i,north lakhimpur,2,laluk,bongalmara,2008-2011
46,assam,i,north lakhimpur,2,laluk,miri gaon,2008-2011
47,assam,i,north lakhimpur,2,laluk,anojuli,2008-2011


In [16]:
final_tehsil_data_multiple_village.to_csv(home_dir.as_posix() + '/data/Tehsil Data/Tehsil_Data_With_Multiple_Village.csv', index=False)

### Unique Tehsils with only 1 Village

In [17]:
grouped = village_list_df.groupby(['State', 'Year'])['Tehsil Name'].nunique().reset_index()

In [18]:
total_unique_tehsils_per_year = grouped.groupby('Year')['Tehsil Name'].sum().reset_index()

In [19]:
total_unique_tehsils_per_year

,Year,Tehsil Name
0,2008-2011,719
1,2011-2014,683
2,2014-2017,791
3,2017-2020,787


In [20]:
single_village_tehsils = village_list_df.groupby(['Tehsil Name', 'Year'])['Village Name'].nunique().reset_index()

In [21]:
single_village_tehsils.sample(10)

,Tehsil Name,Year,Village Name
219,banka,2014-2017,1
1741,mannarkudi,2014-2017,1
2075,palacode,2014-2017,1
2740,thalachery,2008-2011,1
499,chandauli,2008-2011,2
2045,omalur,2014-2017,1
1169,jashipur,2017-2020,1
1101,jabera,2017-2020,1
233,baramati,2014-2017,1
1312,kapadvanj,2017-2020,1


In [22]:
single_village_tehsils = single_village_tehsils[single_village_tehsils['Village Name'] == 1]

In [23]:
single_village_tehsils.head()

,Tehsil Name,Year,Village Name
1,aagar,2008-2011,1
2,aalapur,2014-2017,1
3,aalapur,2017-2020,1
4,aalot,2008-2011,1
5,aamlner,2014-2017,1


In [24]:
final_tehsil_data_single_village = village_list_df[village_list_df['Tehsil Name'].isin(single_village_tehsils['Tehsil Name'])]

In [25]:
final_tehsil_data_single_village.shape 

(2567, 7)

In [26]:
pd.set_option('display.max_rows', None) 

In [27]:
# Find the minimum Tehsil Code for each Tehsil Name
min_tehsil_code = final_tehsil_data_single_village.groupby('Tehsil Name')['Tehsil Code'].transform('min')

# Update the Tehsil Code to be the minimum code for each Tehsil Name
final_tehsil_data_single_village['Tehsil Code'] = min_tehsil_code

/tmp/ipykernel_3597/271811.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tehsil_data_single_village['Tehsil Code'] = min_tehsil_code


In [28]:


# Function to remove rows with duplicate Tehsil Names within each year
def remove_repeating_tehsils(df):
    # Group by Year
    grouped = df.groupby('Year')
    # List to hold the filtered DataFrames
    filtered_dfs = []

    for year, group in grouped:
        # Count occurrences of each Tehsil Name
        tehsil_counts = group['Tehsil Name'].value_counts()
        # Find Tehsil Names that occur more than once
        tehsil_names_to_remove = tehsil_counts[tehsil_counts > 1].index
        # Filter out rows with Tehsil Names that occur more than once
        filtered_group = group[~group['Tehsil Name'].isin(tehsil_names_to_remove)]
        filtered_dfs.append(filtered_group)

    # Concatenate all filtered DataFrames
    result_df = pd.concat(filtered_dfs)
    return result_df

# Apply the function to the DataFrame
df_filtered = remove_repeating_tehsils(final_tehsil_data_single_village)


In [29]:
rows_with_comma = df_filtered['Village Name'].str.contains(',', na=False)

df_filtered = df_filtered[~rows_with_comma]

In [30]:
df_filtered.shape

(2319, 7)

In [31]:
final_tehsil_data_single_village.shape

(2567, 7)

In [ ]:
df_filtered.to_csv(home_dir.as_posix() + '/data/Tehsil Data/Tehsil_Data_With_Single_Village.csv', index=False)